# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [9]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [10]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [11]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [12]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [13]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [14]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [ ]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to delegate the task of continuously monitoring and analyzing market data for AAPL to the Data Analyst in order to identify trends and predict market movements.

Action: Delegate work to co-worker
Action Input: 
{
  "coworker": "Data Analyst",
  "task": "Continuously monitor and analyze market data for AAPL using statistical modeling and machine learning",
  "context": "The goal is to identify trends and predict market movements for AAPL. The Data Analyst should use their expertise in data analysis and statistical modeling to provide insights and alerts about significant market opportunities or threats for AAPL."
}


> Entering new CrewAgentExecutor chain...
I need to gather real-time data on AAPL to identi

Final Answer: Based on the real-time data from Yahoo Finance for AAPL, the stock price is currently at $189.96, showing a 1.65% increase. The market cap is at 2.913T, with a PE ratio of 29.54 and EPS of 6.43. Apple Inc. operates in the consumer electronics and technology sector, offering a range of products and services. The company has seen positive returns over the past year and is a key player in the industry. This information can be used to analyze trends and make informed decisions regarding market movements for AAPL.

> Finished chain.
 

Based on the real-time data from Yahoo Finance for AAPL, the stock price is currently at $189.96, showing a 1.65% increase. The market cap is at 2.913T, with a PE ratio of 29.54 and EPS of 6.43. Apple Inc. operates in the consumer electronics and technology sector, offering a range of products and services. The company has seen positive returns over the past year and is a key player in the industry. This information can be used to analyze trends

Thought: 
I need to delegate tasks to the appropriate team members and ask the necessary questions to ensure we develop the best trading strategies for AAPL.

Action: Delegate work to co-worker
Action Input: {"coworker": "Trading Strategy Developer", "task": "Develop potential trading strategies for AAPL based on data analysis and risk tolerance", "context": "AAPL has shown consistent growth in stock price, positive trends in market cap, PE ratio, and EPS. Data Analyst predicts potential increase in market value due to new products and services. Market volatility and economic conditions can pose threats."} 

I tried reusing the same input, I must stop using this action input. I'll try something else instead.



Action: Ask question to co-worker
Action Input: {"coworker": "Data Analyst", "question": "What specific data points indicate the potential increase in market value for AAPL in the upcoming months?", "context": "AAPL has shown consistent growth in stock price, positive trends in 

Thought: I now can give a great answer

Final Answer: After conducting a search on potential trading strategies for AAPL aligned with the user's risk tolerance and trading preferences, it is evident that individual preferences, risk tolerance, and market understanding play crucial roles in selecting the best strategy. From technical analysis to options trading, there are various approaches available to maximize profits while managing risks effectively. It is recommended to consider the insights provided by reputable sources and tailor the trading strategy to align with the user's specific risk tolerance and preferences.

> Finished chain.
 

After conducting a search on potential trading strategies for AAPL aligned with the user's risk tolerance and trading preferences, it is evident that individual preferences, risk tolerance, and market understanding play crucial roles in selecting the best strategy. From technical analysis to options trading, there are various approaches available t

Final Answer: 
After conducting a detailed analysis of Apple Inc. (AAPL) stock, here are the key insights for potential trading strategies:

- Stock Price Trends: AAPL stock has shown a 1.67% increase today and a 10.75% increase over the past year.
- Market Cap: The market cap is currently at 2.914T, with a 52-week range of 164.08 - 199.62.
- PE Ratio: The PE ratio (TTM) stands at 29.55.
- EPS: The EPS (TTM) is reported at 6.43.

Potential Trading Strategies:
- Considering the positive stock price trends, it may be beneficial to consider a buy-and-hold strategy for AAPL.
- With a healthy PE ratio and EPS, AAPL shows strong fundamental performance, suggesting a long-term investment outlook.
- New product launches could further enhance market value, making AAPL a favorable investment option.
- It is important to monitor market volatility and economic conditions for potential threats that could impact trading decisions.

These insights provide a comprehensive overview of AAPL's current ma

- Display the final result as Markdown.

In [18]:
from IPython.display import Markdown
Markdown(result)

Risk Analysis Report for AAPL:

Introduction:
This report aims to provide a comprehensive analysis of the potential risks associated with the proposed trading strategies and execution plans for AAPL, along with suggested mitigation strategies. The analysis is based on data from reliable sources such as Yahoo Finance, Nasdaq, Bloomberg, and TheStreet.

Market Volatility:
Apple Inc's stock shows relatively low market volatility, with a Sharpe Ratio of 0.0404, indicating a steady performance. The Downside Deviation is 1.42, with a risk-adjusted performance of 0.0259 and a Mean Deviation of 1.12. Apple's beta coefficient is 0.77, suggesting lower volatility compared to the market. Historical and implied volatility are key indicators to consider when evaluating risks associated with trading strategies for AAPL.

Economic Conditions:
Potential threats from market volatility and economic conditions should be factored in when evaluating risks. Mitigation strategies such as using put options can help manage risk exposure to Apple's stock price fluctuations. Considering Apple's market sensitivity, downside risk, and risk-adjusted performance is crucial in aligning trading activities with the firm's risk tolerance.

Mitigation Strategies:
To effectively manage risks associated with the proposed trading strategies and execution plans for AAPL, it is recommended to diversify the portfolio, set stop-loss orders, and use hedging strategies. By tailoring the execution plans to the user's risk tolerance and trading preferences, the team can maximize profits while capitalizing on growth opportunities presented by AAPL.

Conclusion:
In conclusion, a detailed risk analysis of the proposed trading strategies and execution plans for AAPL is essential to mitigate potential risks and optimize trading activities. By considering market volatility, economic conditions, and other factors impacting AAPL's performance, the team can develop effective risk management strategies to achieve the desired outcomes.